In [ ]:
from posydon.config import PATH_TO_POSYDON_DATA
import os

# importing path to our lab data
data_path = os.path.join(os.path.dirname(PATH_TO_POSYDON_DATA), "2025_school_data")

## Background information

The building blocks of POSYDON are the grids of single star and binary MESA simulations.
Since we need large grids for each phase, the ease of setting up MESA simulations is an integral part of the POSYDON infrastructure.

For most population synthesis science cases, the POSYDON default grids might be enough.
But as we start to link more and more population properties to intrinsic stellar and binary physics, additional grids will become more important in our understanding of stars.

So in this practicum we're specifically going to focus on running a "grid" of single stars and binaries, and how you can change their physics.
Because an actual grid run can take several days to complete based on the number of runs, we are going to submit only a few (or a couple) binaries and single stars to show you the process of submission and post-processing.
The main steps in this practicum are:
1. Setting up a grid run with the POSYDON inlists.
2. Adding your own MESA inlist changes on top of the POSYDON ones (this is no MESA summer school though, so nothing too complex)
3. Submitting the grid runs to the cluster.
4. Using pre-ran MESA simulations, we will explore the creation of the `PSyGrid` files, downsampling, and general post-proceesing of these single star and binary runs.


<div class='alert alert-warning'>
For a lot of the things discussed in this notebook, you will have to use the terminal. 
<strong>The terminal commands are written in special markdown cells instead of code cells.</strong>
Make sure to load your POSYDON `conda` environment in the terminal if you're running the commands!
</div>

# A. Setting up the environment for MESA POSYDON runs

Some information about running POSYDON MESA simulations can also be found in the [POSYDON documentation](https://posydon.org/POSYDON/latest/tutorials-examples/MESA-grids/running-grids.html).

First, we have to do some additional setup.
POSYDON MESA requires us to specify the location of the `MESASDK_ROOT`, `MESA_DIR`, and `OMP_NUM_THREADS`, besides the `PATH_TO_POSYDON` and `PATH_TO_POSYDON_DATA`.

Currently, we are using a custom version of r11701 MESA. This means two things:
1. The MESA inlist parameters are a bit older than the online documentation shows.
2. You will need the use the pre-setup MESA environment for this practicum.

MESA requires us to load in a few more variables and initialise the MESASDK in our environment.
For this, we will create a new file that does this for us. Next time you only need to `source` this file to get the MESA installation working.

The file sets:
- `MESA_DIR`, the MESA directory
- `OMP_NUM_THREADS`, the number of CPUs/threads MESA uses
- `MESASDK_ROOT`, the path to the installed MESA SDK

It then initialises the MESASDK to be able to run MESA.

In [ ]:
%%writefile load_MESA
###MESA
# set MESA_DIR to be the directory to which you downloaded MESA
export MESA_DIR=/projects/e33022/MESA/mesa-r11701_witheoschange_andwithreverseMTchange_fiximplicitmdot
# set OMP_NUM_THREADS to be the number of cores on your machine
export OMP_NUM_THREADS=4
# set up the MESA SDK
export MESASDK_ROOT=/projects/e33022/MESA/mesasdk
source $MESASDK_ROOT/bin/mesasdk_init.sh

After running the previous cell, in the terminal, you can now load into your `conda` environment and `source` the load_MESA file.
Make sure you navitate to the folder where the `load_MESA` file is written to, which should be the same directory as the notebook.

```bash

conda activate posydon_env
source load_MESA

```


Check if MESA is correctly loading with

```bash

echo $MESA_DIR

```

If you get the path as above, MESA is correctly loaded.

# 1. POSYDON grid runs configuration file

POSYDON contains the command below to setup a grid run.

```bash
posydon-setup-grid \
    --inifile=HMS-HMS_quest.ini \
    --grid-type=fixed \
    --submission-type=slurm
```
The last two (`grid-type` and `submission-type`) basically do not change at the moment.

The crucial file is the `HMS-HMS_quest.ini`.
POSYDON uses an `.ini` file to setup the MESA inlists for a grid run.
These and the relevant MESA inlists are stored on the [POSYDON-code/POSYDON-MESA-INLIST](https://github.com/POSYDON-code/POSYDON-MESA-INLISTS), where different branches contain different reruns and custom inlists.

We will create a clone of a specific branch (`mb_practicum`) and copy a example config file for the HMS-HMS grid to see how we set this up.


<div class='alert alert-info'>

If you get a warning containing `git`, i.e. `FileNotFoundError: [Errno 2] No such file or directory: 'git'`. 

Please install `git` in your environment with `conda install git`.
</div>


In [ ]:
# Change this to your current working directory
CURRENT_DIR = "CHANGE/TO/YOUR/WORKING/DIRECTORY"

In [ ]:
POSYDON_INLISTS = f"{CURRENT_DIR}/POSYDON-MESA-INLISTS"
!git clone https://github.com/POSYDON-code/POSYDON-MESA-INLISTS.git --branch mb_practicum {POSYDON_INLISTS}

Let's make a new directory and copy the `HMS-HMS_quest` configfile over.

In [ ]:
!mkdir -p {CURRENT_DIR}/run
PATH_TO_YOUR_WORK_DIR = f"{CURRENT_DIR}/run"

In [ ]:
!cp '{POSYDON_INLISTS}/r11701/running_scripts/HMS-HMS_quest.ini' '{PATH_TO_YOUR_WORK_DIR}/HMS-HMS_quest.ini'


This configuration file contains the following sections:

1. `[slurm]`
As the name suggests, this contains all parameters related to the HPC submission manager, slurm.
This includes the number of nodes, to run as a job-array, the partition, user, walltime, and email-notifications.

2. `[mesa_inlists]`
This is where the real setup starts. In this section we determine how POSYDON builds the MESA inlists and what information is stored. We will spend most of this practicum here. 

3. `[mesa_extras]`
This section defines the MESA make files and what `run_star_extras.f` and `binary_star_extras.f` to use.

4. `[run_parameters]`
This section is used to input the location of the `grid.csv` file, which we will discuss later.

Because running MESA models can take a long time, we will not submit a MESA model till the very end. Instead, we will run the "setup" CLI several times to verify if our changes have been applied.

<div class='alert alert-warning'>

**Go into the `HMS-HMS_quest.ini` file in the `run` folder and change the following things:**
    
1. `email` change this to your email address.
2. `posydon_github_root` change this to the `POSYDON-MESA-INLISTS` folder path.

</div>

## `[run_parameters]`

We start our exploration at the last configuration section, because it defines the systems that will be evolved.
`run_parameters` only contains the `grid` parameter, which defined the systems that will be evolved with MESA.

This `grid` parameters needs to point to a CSV file with the following format:

```
initial_z,Zbase,m1,m2,initial_period_in_days
0.00142,0.00142,30.,21.,10.
```

`initial_z` and `Zbase` are the metallicity in absolute metallicity of the model, which are the same here. We will use 1/10th solar here.
The other parameters should speak for themselves with `m1` and `m2` being in solar masses, and `initial_period_in_days` in days (as the name implies). These are specific parameters in the MESA inlists that will be replaced when running the grid.

<div class='alert alert-warning'>

**Comments on Metallicity**

We define the metallicity in multiple places in the setup. The `initial_z` and `Zbase` in the grid are the first location we encounter them. The second location is in the `zams_filename` parameter as part of `[mesa_inlists]` later in the tutorial. **The metallicity and composition of the selected ZAMS model and the grid file need to be the same.** Otherwise, MESA will not be able to evolve the star! For this practicum, you should not have to change the `zams_filename`.
</div>


In [ ]:
import csv
import os

test_file = 'grid_test.csv'
path_to_file = os.path.join(PATH_TO_YOUR_WORK_DIR, test_file)

with open(path_to_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['initial_z','Zbase','m1','m2','initial_period_in_days'])
    # Feel free to change the binary parameters: 
    # m1, m2, and initial_period_in_days here to test a different system
    # Please don't change the first two columns for this practicum.
    writer.writerow([0.00142, 0.00142, 30., 21., 10.])


## `[mesa_inlists]`

### `job` & `controls` 
This is the heart of the POSYDON MESA inlist building.
Three levels of `inlist`'s can be defined:
1. The MESA r11701 default values: `binary_controls_mesa_defaults`
2. The POSYDON inlist: `binary_controls_posydon_defaults`
3. User defined inlists: `binary_controls_user`

These are stacked `1,2,3` with the latest being load in last and overwritting any previously defined variables.

In the configuration file, we are currently only defining the `mesa_defaults` values

### `columns`

POSYDON requires special columns to be present in the history, binary and profile for the post-processing.
As such, we do not use the mesa defaults and instead directly point to the `POSYDON-MESA-INLISTS` to get the POSYDON default.
There is no stacking occuring for the columns definition.
Of course, you can alter the path and point to your own columns file(s).


### additional parameters

There are a few more parameters in this section:

1. `scenario`, which is currently commented out. We will discuss at the end of the practicum.
2. `zams_filename`. As the name suggests, this file contains multiple ZAMS models at different masses for a specific metallicity configuration.
POSYDON contains 8 different metallicity ZAMS models. If you change the metallicity in the grid file, the ZAMS model also needs to be changed!
3. `single_star_grid`. Used to run a single star grid.

### MESA output controls

These parameters describe what data is stored from the run.
You should not have to change these.

## `[mesa_extras]`

This section defines the `run_star_extras` and `run_binary_extras` used in the simulation. POSYDON uses a specific `run_star_extras` in the binary runs with additional termination flags for the post-processing.

`mesa_binary_extras` and `mesa_star_binary_extras` define what files are used in the binary runs.

`mesa_star1_extras` and `mesa_star2_extras` are used for single stars and pre-MS evolution, such as the CO-HeMS grid.
In the HMS-HMS grid run we are doing here, these are copied but not used in the evolution.




# 2. Building MESA with POSYDON

Let's go into our run directory and build the inlists using only the MESA defaults.
Run the following commands in your terminal session:

```bash
cd run

posydon-setup-grid \
    --inifile=HMS-HMS_quest.ini \
    --grid-type=fixed \
    --submission-type=slurm
```

## `[mesa_extras]`

This will build MESA for you based on the configuration file and all the files needed to submit our one binary to the cluster.
In this basic form, we are using the MESA default parameter, `run_star_extras`, `run_binary_extras`, and only the POSYDON columns.

<details>

<summary> <strong>Details on the produced files</strong> </summary>

A bunch of files and folders are created when you ran the previous command.
In short, this is what each of them does:

- binary: contains the MESA binary components
- columns_lists: contains the MESA columns configuration
- star1:  contains the MESA star1 components
- star2:  contains the MESA star2 components
- cleanup.slurm: compresses the MESA output.
- job_array_grid_submit.slurm: submits the job array to the cluster
- mk: makes MESA (already done)
- run_grid.sh: submit the `job_array_grid_submit.slurm` and `cleanup.slurm` to the cluster.

</details>

Let's make sure we have just the basic MESA inlists and `run_star_extras.f`.
Opening up the `binary/src/run_star_extras.f` should show a file with ~41 lines of code.


```bash

cat binary/src/run_star_extras.f

```


This shows that our `posydon-setup-grid` has correctly ran.

However, if we submitted something now we would just get a basic MESA binary run without 
the POSYDON physics choices. We would like to select our own inlists,`run_star_extras.f` and `binary_extras.f`.

<div class='alert alert-success'>

Let's uncomment the `user_binary_extras` and `user_star_binary_extras` in the `HMS-HMS_quest.ini` configuration file.
These new variables will overwrite the `mesa` defined variables.

Now rerun `posydon-setup-grid`.
</div>

You should see the a `ReplaceValueWarning` showing up with `ReplaceValueWarning: 'Section mesa_extras value mesa_binary_extras is being set to None'`. This indicates that the the `user` configuration is used instead.

Let's see if the `run_star_extras.f` and `run_binary_extras.f` now contains more code.

```bash
cat run/binary/src/run_star_extras.f
cat run/binary/src/run_binary_extras.f
```

## `[mesa_inlists]`

Surpise! It does! We are now loading in the POSYDON MESA default binary and star extras!
Let's now also make sure the POSYDON inlists are correctly loaded.

Let's check what the current state of `inlist_project`, `inlist1` or `inlist2` is:

<div class='alert alert-info'>

If you see `initial_mass= 1` or `m1 = 1.0d0`, <strong>don't be alarmed</strong>. We replace the initial values for each binary when we submit the grid using `posydon-run-grid`. This happens behind the scenes, as long as `grid_test.csv` is correctly created. If you see `Grid parameters that effect binary_controls: m1,m2,initial_period_in_days` in the output of the `posydon-setup-grid`, everything is working correctly.
</div>

```bash
cat run/binary/inlist_project
```


<div class='alert alert-success'>

Ignoring the `Star Formation` parameters, uncomment the other `posydon_defaults` parameters in the `HMS-HMS_quest.ini` file, for example `binary_controls_posydon_defaults`, `binary_jobs_posydon_defaults`, etc.. The POSYDON files contain both `_control` and `_job` components, thus they point to the same files.
But you're not required to keep them in the same file.

Then rerun `posydon-setup-grid`.

</div>

We're changing too many parameters to list them, we show those in the beginning of 
`inlist_project` (in `&binary_controls`) and `inlist1` (`&controls`) files below.

<details>

<summary> `inlist_project` example differences </summary>

![Difference between MESA and POSYDON default inlist_project](./diff-mesa-posydon-inlist_project.png)

</details>

<details>

<summary> `inlist1` example differences </summary>

![Difference between MESA and POSYDON default inlist1](./diff-mesa-posydon-inlist1.png)

</details>

We have now reached a default POSYDON setup!

## Custom user input

The last layer is the `user` defined files, where you can define your own parameters on top of the MESA and POSYDON inlists.

<div class='alert alert-success'>

### Exercise

Running binary models can take a long time, therefore you are going to introduce an artificial stopping condition for the binary:

By creating a custom `user` MESA inlist and loading it into the grid building, can you stop the model at `model_number` 500?

<details>
<summary><strong>Hint</strong></summary>

`max_model_number` can be defined in the `star` controls, which stops the evolution after `X` models.

</details>

</div>


<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>
Write into a file:

```
&star_controls
    max_model_number = 500
/ ! end user controls
```

Then point to this file for your user star and binary controls inlists.
</details>

## Submitting the grid

Now that we have set up the grid, it's time to submit it to the cluster!
`posydon-grid-setup` created a `run_grid.sh` script for us, which is the only thing we have to run. `./run_grid.sh` or `bash run_grid.sh` will submit the MESA simulations to the HPC.
This will also automatically submit a second job which compresses the output files to reduce the size they take up.

<div class='alert alert-success'>

### Exercise

**Submit the grid!**

Well done you've now submitted a first "grid" to the cluster using POSYDON! Keep an eye on the run during the next part of the tutorial. You should be able to see if the simulation started and if reasonable output is/will be produced.

</div>



## Additional material on grid set up

There are a few parameters we haven't discussed in detail. We do not have the time to go into them for this practicum, but please ask one of the developers if you want to run a specific setup and want help with that.

### The `scenario` parameter

To allow for reproducible science, we have also added the `scenario` parameter.
It allows you to setup a specific branch and commit of the POSYDON-MESA-INLISTS github repository to use for the grid.
The `scenario` parameter consists of 3 different parameters:

1. `posydon` indicating what repository to use.
2. `mb_practicum-6968c2a2b0f0fad05da8cab1055d30ab3fd3d58a`, where `mb_practicum` is the branch name and `6968c2...` the commit you want to use. 
3. `HMS-HMS` defines what grid you want to run.

Although you can no longer define the `posydon` parameters when using the scenario. You are still able to overwrite paramters with the `user` inputs.

### Additional Parameters

We have not really touched upon a few parameters in the POSYDON grid config file, for example, `single_star_grid` or `star1/2_formation_job`.

- `single_star_grid` is used for running a single star grid and uses the single star termination conditions, which is especially useful when using a `scenario`. For example, using the `HMS-HMS` tag with `single_star_grid=True` will run H-rich single star models.

- The `star1/2_formation_job`s are for creating the ZAMS models and for creating the He stars in the CO-HeMS grids. 


### Local testing

Sometimes you would like to know if a binary will run correctly or only want to run a very small grid locally. 
You can replace `--submission-type=slurm` with `--submission-type=shell` to run something locally. This will create `grid_command.sh`, which runs MESA in the local shell.


# 3. Post-processing a grid

Once binaries have finished running, you will end up with a collection of folders in your run directory, each being a binary (or single star) simulation. Because running a binary model to its actual termination conditions can take up to two days, we have provided a few example binaries to work with.

In `$PATH_TO_POSYDON_DATA/2025_school_data/MESA-models` you can find two sets of 10 binaries.
These are two "runs" of HMS-HMS runs from different MESA grid runs. This data structure with a different binary per folder in different can be hard to navigate and add extra information to, such as the core-collapse outcome, etc. That's where the `PSyGrid` comes in. We have already worked with the POSYDON provided ones in the previous practicum, but now we will create our own. Either through the scripts in POSYDON or manually.

The post-processing consists of four main steps:
1. `step_1`: create a `PSyGrid` object.
2. `step_2`: combined `PSyGrid` objects.
3. `step_3`: calculate extra values.
4. `step_F`: export dataset

There are additional steps available for training the interpolators:
- `step_4`: trains interpolators
- `step_5`: trains the profile interpolators.

Sometimes MESA require more fine-tuning of the numerical modelling to converge on a solution.
However, you do not want to rerun the complete grid again, especially if only numerical changes are made.
For rerunning part of the grid, there's also the actual final step:
- `rerun`: exports model parameters that need to be rerun based on a rerun selection.

In this practicum, we will focus on `step_1` to `step_4`. 

## `step_1` create a `PSyGrid` object

Without defining a path on the instance creation, we can initialise our `PSyGrid` instance.

In [ ]:
from posydon.grids.psygrid import PSyGrid

grid = PSyGrid(verbose=True)

MESA_grid_path = os.path.join(data_path, "MESA-models")

We start with `grid_run_1` and define a grid name for the `PSyGrid` file.

In [ ]:
PATH_TO_YOUR_WORK_DIR = os.getcwd()
grid_1 = f"{MESA_grid_path}/grid_run_1/"
psygrid_path_1 = f"{PATH_TO_YOUR_WORK_DIR}/grid_1.h5"

In [ ]:
grid.create(grid_1,
            psygrid_path_1,
            overwrite=True,
            initial_RLO_fix=True)

<div class='alert alert-success'>

### Exercise

1. Repeat the process for `grid_runs_2`.
2. Load one of the newly created `PSyGrid` files. Go back to practicum 1, if you need a reminder on that.
3. Check what information is available in this newly created `PSyGrid` object. Are the `initial_values` there? Are all the `final_values` there?
4. Do you notice anything different in the timeseries compared to the grids in the practicum 1?
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

```
grid_2 = f"{MESA_grid_path}/grid_run_2/"
psygrid_path_2 = f"{PATH_TO_YOUR_WORK_DIR}/grid_2.h5"
grid.create(grid_2,
            psygrid_path_2,
            initial_RLO_fix=True)
```
</details>


## `step_2` combining grids

The population synthesis in POSYDON needs a single grid file as an input for an evoltuionary step.
So we need to combined the grids into a single file.
This is also useful when you have rerun a model which previously failed, allowing you to replace the old failed model with a new, hopefully succesful, model.

<div class='alert alert-success'>

### Exercise

Combine your newly created grids into a single grid file.

</div>


In [ ]:
from posydon.grids.psygrid import join_grids

join_grids(input_paths= [# fill in],
           output_path= # fill in
           verbose=True)


<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

```
from posydon.grids.psygrid import join_grids
join_grids(input_paths=[psygrid_path_1, psygrid_path_2],
            output_path='./combined.h5',
            verbose=True)
```

</details>

## `step_3` add extra values.

As one of the previous exercise hinted towards, the `SN_MODELS` are missing in this new `PSyGrid` object.
These are added in this third post-processsing step using the build-in `post_process_grid` function.
Through this process you can manually add any more columns if you like.

In [ ]:
from posydon.grids.post_processing import post_process_grid
from posydon.grids.SN_MODELS import SN_MODELS
MESA_dir_names, data = post_process_grid(grid=grid,
                                index=None,
                                star_2_CO=False,
                                SN_MODELS=SN_MODELS)

`data` contains a whole bunch of additional columns that can be added to the grid, but they have not yet been added to the grid itself. You can see here as well that the `SN_MODELS` are automatically calculated in the data. This is an optional input, but you can change or add your own supernova models to it.
For the output of `post_process_grid`, we have the additional function `add_post_processed_quantities`.

This looks over all the new variables and adds them to the MESA grid in similar fashion as you've seen in practicum 1.

In [ ]:
from posydon.grids.post_processing import add_post_processed_quantities

add_post_processed_quantities(grid=grid,
                              MESA_dirs_EXTRA_COLUMNS=MESA_dir_names,
                              EXTRA_COLUMNS=data)

# This is similar to manually running, though the MESA_dir_names are not added.
for name, values in zip(data.keys(), data.values()):
    grid.add_column(name, values, overwrite=True)

<div class='alert alert-success'>

### Exercise

In an earlier practicum, you worked with the `SN_MODELS`. Again we're going to pick a model and change a few things.
1. Pick `SN_MODEL_v2_0` and change the `PISN_CO_shift` downwards with -30. This is an extreme value and not physically motivated.
2. Get the new output columns using `post_process_grid`.
3. Add the new columns to the grid file.

If you have the time, how would you verify if the change has been implemented?
</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

```
from posydon.grids.SN_MODELS import get_SN_MODEL, SN_MODELS

new_MODEL = get_SN_MODEL(list(SN_MODELS.keys())[0])

new_MODEL['PISN_CO_shift'] += 10.0
print(new_MODEL)

MESA_dir_names, data = post_process_grid(grid=grid,
                                index=None,
                                star_2_CO=False,
                                SN_MODELS=new_MODEL)

```

</details>

Now you should be able to add you own supernova models from your MESA runs!

### Downsampling and compression

You might have noticed that the MESA time in the POSYDON grids from practicum 1 and the `PSyGrid` you've just created are quite different.
This is because you've just created lossless versions of the MESA runs, which contains all of the MESA run information, including the full profiles etc.

While this is great for local working, the PSyGrid object are already staring to become a few megabytes with only 20 MESA runs.
For large MESA grids, this quickly becomes unmanagable and difficult to distribute.
As such, we have implemented a downsampling, which reduces the data size by removing datapoints if the change between points is small.

This downsampling is done when you first create the `PSyGrid` objects in `step_1`.
We define this grid as a `LITE` grid, while the lossless compressing is named `ORIGINAL` (so original, I know :) )
Both the timeseries and profiles are downsampled in this configuration.

The downsampling of the profile can lead to inconsistencies in the spin if you use it to calculate a collapse into a compact object.
As such, in POSYDON, we use the `ORIGINAL` grids to do the supernova and core-collapse, but add these parameters to the downsampled `LITE` grids. Thus, giving us the best of both worlds. The detailerd profile for the remnant mass calculation and being able to distribute the large grids.

In [ ]:
from posydon.grids.psygrid import (PSyGrid,
                                   DEFAULT_HISTORY_DS_EXCLUDE,
                                   DEFAULT_PROFILE_DS_EXCLUDE,
                                   EXTRA_COLS_DS_EXCLUDE)
COMPRESSIONS = {
    'ORIGINAL' : {
        'history_DS_error'    : None,
        'profile_DS_error'    : None,
        'profile_DS_interval' : None,
        'history_DS_exclude'  : DEFAULT_HISTORY_DS_EXCLUDE,
        'profile_DS_exclude'  : DEFAULT_PROFILE_DS_EXCLUDE
    },
    'LITE' : {
        'history_DS_error'    : 0.1,
        'profile_DS_error'    : 0.1,
        'profile_DS_interval' : -0.005,
        'history_DS_exclude'  : EXTRA_COLS_DS_EXCLUDE,
        'profile_DS_exclude'  : EXTRA_COLS_DS_EXCLUDE
    }
}

psygrid_path_1_LITE = f"{PATH_TO_YOUR_WORK_DIR}/grid_1_LITE.h5"
grid_LITE = PSyGrid(verbose=True)
grid_LITE.create(grid_1,
            psygrid_path_1_LITE,
            overwrite=True,
            history_DS_error=COMPRESSIONS['LITE']['history_DS_error'],
            profile_DS_error=COMPRESSIONS['LITE']['profile_DS_error'],
            history_DS_exclude=COMPRESSIONS['LITE']['history_DS_exclude'],
            profile_DS_exclude=COMPRESSIONS['LITE']['profile_DS_exclude'],
            profile_DS_interval=COMPRESSIONS['LITE']['profile_DS_interval'],
            compression="gzip9",
            initial_RLO_fix=True)

<div class='alert alert-success'>

### Exercise

It is up to you know. Can you create a downsampled grid which:
1. Combines both grid runs into one file.
2. Uses the non-downsampled grid to calculate the SN_MODELS and add those columns to the downsampled grid file?

</div>

## step_4 training the interpolators and classifiers

POSYDON comes with the possibility to train interpolators and classifiers to more quickly evolve binaries, but also to populate the parameter space between MESA model grid points. On Friday, you will see more about this, but in our post-processing is where we train these models.

There are 3 steps involved in training the Initial-Final interpolators.
1. Set up.
2. Training
3. Saving

During the setup, we need to define what interpolators to use and on what classes you want the interpolators to be conditioned on.
The interpolator and classifier training requires the following input type:

```python

interpolators = [
        {
            "interp_method": interp_methods,
            "interp_classes": interp_classes,
            "out_keys": out_keys,
            "out_nan_keys": out_nan_keys,
            "class_method": "kNN",
            "c_keys": c_keys,
            "c_key": "interpolation_class" # note the difference c_keys vs c_key
        },
    ]

```

This takes care of two things at the same time:
1. We will be training a classifier on categorial data, these are the `"c_keys"`. The classifier being trained is a k-nearest neighbour (`"class_method": "kNN`). For example, think of these as the mass transfer stability or SN outcome.
2. We are training an interpolator that predicts the final values (`"out_keys"`) based on the input binary parameters of the grid. We create multiple interpolators (`"interp_method"`) based on the classes (`"interp_classes"`) inside a specific key (`"c_key"`). `"out_nan_keys"` are what categories to set to NaN as an output.

For the HMS-HMS grid, as in our example here, we use 4 classes.


In [ ]:
interp_methods = ['linear', 'linear', 'linear', 'linear']
interp_classes = ['stable_MT', 'unstable_MT', 'no_MT', 'stable_reverse_MT']

<div class='alert alert-warning'>

The classification requires quite a few number of data points in the grid to run.
Our example poulation above is insufficient to properly train the classification and thus the interpolator.

Instead, we will copy one grid out of `$PATH_TO_POSYDON_DATA` grids and use it to train a new interpolator.
</div>

We use all the keys in the `"final_value"` that are not a SN model or categorial data (strings).

In [ ]:
from posydon.config import PATH_TO_POSYDON_DATA
from posydon.grids.psygrid import PSyGrid

In [ ]:
!cp "{PATH_TO_POSYDON_DATA}/HMS-HMS/1e-01_Zsun.h5" .

In [ ]:
import numpy as np

path_to_grid = "./1e-01_Zsun.h5"
grid = PSyGrid(path_to_grid)

out_keys = [key for key in grid.final_values.dtype.names if (
                                   key != "model_number" and
                                   (type(grid.final_values[key][0]) != np.str_)
                                   and any(~np.isnan(grid.final_values[key]))
                                   and "SN_MODEL" not in key)]
print(out_keys)

A crucial aspect is to make sure that output values that are all None or NaN are not trained on, since
there's no useful info in those parameters.

In [ ]:
out_nan_keys = [key for key in grid.final_values.dtype.names if (
                             key != "model_number" and
                            (type(grid.final_values[key][0]) != np.str_)
                            and all(np.isnan(grid.final_values[key]))
                            and "SN_MODEL" not in key)]

Finally, we define any categorial data we want to train at the same time:

In [ ]:
c_keys = ['interpolation_class', 'S1_state', 'S2_state', 'mt_history']
for SN_MODEL_NAME in SN_MODELS.keys():
    for i in range(1,3):
        c_keys.append(f'S{i}_{SN_MODEL_NAME}_SN_type')
        c_keys.append(f'S{i}_{SN_MODEL_NAME}_CO_type')

That's it. We now have all the parameters required to train the interpolators on the MESA grid.
The interpolator will take the initial values of a binary, determine its "interpolation" class using the classifier, and then use the relevant interpolator based on this class. This interpolator will return the properties of the star at carbon-depletion.

In [ ]:
interpolators = [
        {
            "interp_method": interp_methods,
            "interp_classes": interp_classes,
            "out_keys": out_keys,
            "out_nan_keys": out_nan_keys,
            "class_method": "kNN",
            "c_keys": c_keys,
            "c_key": "interpolation_class"
        },
    ]

In [ ]:
from posydon.interpolation.IF_interpolation import IFInterpolator

interp = IFInterpolator(grid=grid,
                        interpolators=interpolators)

In [ ]:
# training and saving
interp.train()
interp.save("./IFinterpolator.pkl")

Well done! You now have created your first interpolators. These can now be used in POSYDON to interpolate the binary evolution and expand the parameter space coverage.

<div class='alert alert-success'>

### Exercise

The interpolator you've trained only works on the binary evolution part. If this is all you're interested in that's fine.
But we can go further, we can also include the core-collapse models as something to interpolate over.

For each `"SN_MODEL"`, we need to create a new interpolator and add it to our interpolators.

</div>

In [ ]:
for SN_MODEL_NAME in SN_MODELS.keys():
    for i in range(1,2):
        out_keys = # ?
        
        out_nan_keys = # ?
        
        # get interpolations classes dynamically
        interp_method = []
        interp_classes = []
        for CO_interpolation_class in ["BH", "NS", "WD", "BH_reverse_MT"]:
            if CO_interpolation_class in grid.final_values[f'S{i}_{SN_MODEL_NAME}_CO_interpolation_class']:
                interp_method.append("linear")
                interp_classes.append(CO_interpolation_class)

        one_interpolator = {
                "interp_method": interp_method,
                "interp_classes": interp_classes,
                "out_keys": out_keys,
                "out_nan_keys": out_nan_keys,
                "class_method": "kNN",
                "c_keys": [f'S{i}_{SN_MODEL_NAME}_CO_interpolation_class', 
                            f'S{i}_{SN_MODEL_NAME}_CO_type',
                            f'S{i}_{SN_MODEL_NAME}_SN_type'],
                "c_key": f'S{i}_{SN_MODEL_NAME}_CO_interpolation_class'
            },

        interpolators.append(one_interpolator)

In [ ]:
interp = IFInterpolator(grid=grid,
                              interpolators=interpolators)
interp.train()
interp.save("./IFinterpolator.pkl")

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

```python
for SN_MODEL_NAME in SN_MODELS.keys():
    for i in range(1,2):
        out_keys = [key for key in grid.final_values.dtype.names if (
                                   key != "model_number" and
                                   (type(grid.final_values[key][0]) != np.str_)
                                   and any(~np.isnan(grid.final_values[key]))
                                   and f"S{i}_{SN_MODEL_NAME}" in key)]
        
        out_nan_keys = [key for key in grid.final_values.dtype.names if (
                                key != "model_number" 
                                and (type(grid.final_values[key][0]) != np.str_)
                                and all(np.isnan(grid.final_values[key]))
                                and f"S{i}_{SN_MODEL_NAME}" in key)]
        
        # get interpolations classes dynamically
        interp_method = []
        interp_classes = []
        for CO_interpolation_class in ["BH", "NS", "WD", "BH_reverse_MT"]:
            if CO_interpolation_class in grid.final_values[f'S{i}_{SN_MODEL_NAME}_CO_interpolation_class']:
                interp_method.append("linear")
                interp_classes.append(CO_interpolation_class)

        one_interpolator = {
                "interp_method": interp_method,
                "interp_classes": interp_classes,
                "out_keys": out_keys,
                "out_nan_keys": out_nan_keys,
                "class_method": "kNN",
                "c_keys": [f'S{i}_{SN_MODEL_NAME}_CO_interpolation_class', 
                            f'S{i}_{SN_MODEL_NAME}_CO_type',
                            f'S{i}_{SN_MODEL_NAME}_SN_type'],
                "c_key": f'S{i}_{SN_MODEL_NAME}_CO_interpolation_class'
            },

        interpolators.append(one_interpolator)

```


</details>

Now you have an interpolator which even interpolates over the SN model!
These additional interpolated values will be used with the `linear3c` interpolation method for `step_SN` if `use_interp_values=True`.

<div class='alert alert-warning'>

Well done! You've just created everything you need to run your grid with POSYDON.

</div>

# POSYDON post-processing configuration file

As you might have notices, there are a lot of tedious steps to take in post-processing to get the data products
required for the POSYDON population synthesis.
If you're initially setting things up, this might be useful for debugging.
For production of the grids, we have simplified this setup into a configuration file similar to the MESA grid runs.

These tutorials describe this in more details:
- [Processing MESA grids with the POSYDON pipeline API](https://posydon.org/POSYDON/latest/tutorials-examples/generating-datasets/just_step_1.html)
- [Full pipeline](https://posydon.org/POSYDON/latest/tutorials-examples/generating-datasets/run_full_pipeline.html)

In [ ]:
# let's download this configuration file
import urllib
url = "https://raw.githubusercontent.com/POSYDON-code/POSYDON/refs/heads/main/grid_params/pipeline_quest.ini"

urllib.request.urlretrieve(url, 'pipeline_quest.ini')

If you open up the file, you will notice multiple sections again. We won't run this now. Instead as a closing remanrk, we will describe how these parameters link back to how the previous steps you've done.

- `[account]` describes the `slurm` setup parameters

- `[pipeline setup]` describes what steps will be ran in the pipeline, what verification plots will be made, etc.

- `[step_1]`, `[step_2]`, etc.
These steps describe what grids are created or combined at each step.
You can process multiple grid types, metallicity and slices at the same time.

In `[step_1]`, we create the `PSyGrid` object and their compression.
Each entry in `GRID_SLICES` is a grid run, like your `grid_run_1` and `grid_run_2`. Note that the each `GRID_TYPES` entry links to the list in the list of `GRID_SLICES`.

In `[step_2]`, we combine the grid slices together.
`GRID_SLICES` combines the previous created `PSyGrid` into new combined grids. If you already have created the `PSyGrid` object, you could even skip `[step_1]`.

While the initial setup of this configuration file might be more work initially, it will become easy to add additional grids and different stacking of grids too.